# install packages

In [1]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import os
import pickle

In [4]:
%run ./WalletClustering_neo4jConnect.ipynb # includes Neo4J connector
# methods & variables of notebook can be referenced

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# query terror addresses

In [5]:
#The below code is returning a list of all the addresses that are marked as terror addresses.
#Query takes around 10 min
def returnTerrorAddresses():

    query_string = '''
    MATCH (a:Address {isTerror: True})
    Return a.address
    '''

    response = conn.query(query_string, db='neo4j')
    terrorAddresses = [r[0] for r in response]
    return terrorAddresses

In [6]:
# comment out entire block if old terrorAddressList saved as pickle should be used
def createTerrorAddressList():
    terrorAddressList = returnTerrorAddresses()

    # save terrorAddressList to file
    with open('terrorAddressList.pickle', 'wb') as export:
        pickle.dump(terrorAddressList, export)

In [7]:
# use existing blacklist if exists
if not os.path.exists('terrorAddressList.pickle'):
    createTerrorAddressList()

terrorAddressList = pickle.load(open('terrorAddressList.pickle', 'rb'))

# remove unwanted addresses

In [8]:
# # allows blacklist of addresses in json format. json file retrieved with https://github.com/nicotom/walletexplorer_scrapy
# 
# def createBlacklist():
#   dirname = os.path.dirname(os.path.realpath('__file__'))
#   folder = os.path.join(dirname, 'exchange_addresses\\json\\')
#   
#   def flatten(t):
#       return [item for sublist in t for item in sublist]
#   
#   blacklistAddresses = []
#   
#   # iterate over collection of exchange addresses in csv format to create blacklist
#   # attention, lists are not complete yet!
#   
#   for file in os.listdir(folder):
#       df = pd.read_json(os.path.join(folder, file))
#       blacklistAddresses.append(df['addresses'].tolist())
#       continue
#   
#   flatten(flatten(blacklistAddresses))

In [9]:
# flattens list of lists
def flatten(list):
    return [item for sublist in list for item in sublist]

In [10]:
# allows blacklist of addresses in csv format
blacklistAddresses = []
def createBlacklist():
    dirname = os.path.dirname(os.path.realpath('__file__'))
    folder = os.path.join(dirname, 'exchange_addresses\\csv\\')
   
    # iterate over collection of exchange addresses in csv format to create blacklist
    # attention, lists are not complete yet!
    for file in os.listdir(folder):
        df = pd.read_csv(os.path.join(folder, file), header=0)
        blacklistAddresses.append(df['hashAdd'].tolist())
        continue
    
    flatten(blacklistAddresses)

    # append known exchange addresses outside large dataset
    # remember to delete blacklistAddresses.pickle after adding addresses to recompute blacklist
    blacklistAddresses.append('13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc')
    blacklistAddresses.append('1EP391qsP7QUfqhnziAiFgY8pMDtvmB9i6')
    blacklistAddresses.append('1FGhgLbMzrUV5mgwX9nkEeqHbKbUK29nbQ')
    blacklistAddresses.append('19ckaZLPDzn7s2EcFtsjLobu71eAqWFDD8')
    
    
    #export blacklistAddresses to file
    with open('blacklistAddresses.pickle', 'wb') as export:
        pickle.dump(blacklistAddresses, export)

In [11]:
# use existing blacklist if exists
if not os.path.exists('blacklistAddresses.pickle'):
    createBlacklist()

blacklistAddresses = pickle.load(open('blacklistAddresses.pickle', 'rb'))

In [12]:
# to be used for removal of exchange addresses
def addressCleanUp(addressList, blacklist):
    interestingAddresses = []
    for address in addressList:
        if address not in blacklist:
            interestingAddresses.append(address)
    return interestingAddresses

In [13]:
def createAddressesToClusterList():
    addressesToCluster = addressCleanUp(terrorAddressList, blacklistAddresses)

    # save addressesToCluster to file
    with open('addressesToCluster.pickle', 'wb') as export:
        pickle.dump(addressesToCluster, export)

In [14]:
# load addressesToCluster if exists
if not os.path.exists('addressesToCluster.pickle'):
    createAddressesToClusterList()

addressesToCluster = pickle.load(open('addressesToCluster.pickle', 'rb'))